In [5]:
!pip install pandas

In [44]:
pip install torch torchvision


     ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
     - -------------------------------------- 0.0/1.2 MB 1.3 MB/s eta 0:00:01
     ---- ----------------------------------- 0.1/1.2 MB 1.8 MB/s eta 0:00:01
     ------ --------------------------------- 0.2/1.2 MB 1.6 MB/s eta 0:00:01
     ---------- ----------------------------- 0.3/1.2 MB 2.0 MB/s eta 0:00:01
     ---------------- ----------------------- 0.5/1.2 MB 2.4 MB/s eta 0:00:01
     ------------------ --------------------- 0.6/1.2 MB 2.5 MB/s eta 0:00:01
     --------------------- ------------------ 0.6/1.2 MB 2.3 MB/s eta 0:00:01
     --------------------------- ------------ 0.8/1.2 MB 2.6 MB/s eta 0:00:01
     -------------------------------- ------- 1.0/1.2 MB 2.8 MB/s eta 0:00:01
     ------------------------------------- -- 1.1/1.2 MB 2.9 MB/s eta 0:00:01
     ---------------------------------------- 1.2/1.2 MB 2.8 MB/s eta 0:00:00
  Obtaining dependency information for pillow!=8.3.*,>=5.3.0 from http

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [2]:
import nltk
import numpy as np

nltk.download('punkt')
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()


def tokenize(sentence):
    return nltk.word_tokenize(sentence)


def stem(word):
    return stemmer.stem(word.lower())


def bag_of_words(tokenized_sentence, all_words):
    """
    Example:
    sentence = ["hello", "how", "are", "you"]
    words = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
    bag =   [  0,   1,       0,    1,    0,     0,        0]
    """
    tokenized_sentence = [stem(w) for w in tokenized_sentence]
    bag = np.zeros(len(all_words),dtype = np.float32)
    for index, w in enumerate(all_words):
        if w in tokenized_sentence:
            bag[index] = 1.0
    return bag

sentence = ["hello", "how", "are", "you"]
words = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
bag = bag_of_words(sentence,words)
print(bag)
            


#use of tokenization
a = "How are you doing?"
print(a)
a = tokenize(a)
print(a)

#use of stemming
words = ["Organize", "organizes", "Organizing"]
stemmed_words = [stem(w) for w in words]
print(stemmed_words)


[0. 1. 0. 1. 0. 0. 0.]
How are you doing?
['How', 'are', 'you', 'doing', '?']
['organ', 'organ', 'organ']


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Raghav\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
import pandas as pd
import numpy as np

In [4]:
import json

In [5]:
df = pd.read_json('h.json')

In [6]:
with open('h.json', 'r') as f:
    intents = json.load(f)

print(intents)
    

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'Hey', 'How are you', 'Is anyone there?', 'Hello', 'Good day', 'Hello,I want to ask some questions'], 'responses': ['Hey :-)', 'Hello, thanks for visiting', 'Hi there, what can I do for you?', 'Hi there,how can I help?', "Hello sir/ma'am,Welcome!", "Hey,I'm mike, What can I do for you? "]}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye', "You're great,see you soon", 'Thank you,great help'], 'responses': ['See you later,thanks for visiting', 'Have a nice day', 'Bye! Come back again soon', 'Pleasure to solve your queries,come back again soon', 'Great to interact with you,Goodbye']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", "Thank's a lot"], 'responses': ['Happy to help!', 'Any time!', 'My pleasure']}, {'tag': 'items', 'patterns': ['Which items do you have?', 'What kind of items are there?', 'What do you sell?', 'What are the items do you sell?'], 'responses': ['We sell coffee and tea', '

In [7]:
all_words = []
tags = []
xy = []

for intent in intents['intents']:
    tag = intent['tag']
    #add to tags list
    tags.append(tag)
    for pattern in intent['patterns']:
        #tokenize each word in the sentence
        w = tokenize(pattern)
        #add to our words list
        all_words.extend(w)
        #add to xy pair
        xy.append((w,tag))

ignore_words =  ['?','!','.',',']
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
print(all_words)
tags = sorted(set(tags))
print(tags)
        
        
        


["'re", "'s", 'a', 'accept', 'anyon', 'are', 'ask', 'bye', 'can', 'card', 'cash', 'coff', 'cold', 'credit', 'day', 'deliveri', 'do', 'doe', 'get', 'good', 'goodby', 'great', 'have', 'hello', 'help', 'hey', 'hi', 'hot', 'how', 'i', 'is', 'item', 'kind', 'later', 'long', 'lot', 'mastercard', 'my', 'of', 'onli', 'or', 'pay', 'paypal', 'question', 'see', 'sell', 'ship', 'some', 'soon', 'take', 'thank', 'that', 'the', 'there', 'to', 'type', 'upi', 'want', 'what', 'when', 'which', 'with', 'you']
['delivery', 'goodbye', 'greeting', 'items', 'payments', 'thanks', 'types']


In [8]:
X_train = []
Y_train = []
for(pattern_sentence,tag) in xy:
    bag = bag_of_words(pattern_sentence,all_words)
    X_train.append(bag)

    label = tags.index(tag)
    Y_train.append(label)

X_train = np.array(X_train)
Y_train = np.array(Y_train)

In [9]:
class NeuralNet(nn.Module):
    def __init__(self,input_size,hidden_size,num_classes):  #feed forward neural network
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size,hidden_size)
        self.l2 = nn.Linear(hidden_size,hidden_size)
        self.l3 = nn.Linear(hidden_size,num_classes)
        self.relu = nn.ReLU()

    def forward(self,x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        #after l3 no activation function and no softmax because we are going to apply crossentropyloss
        return out
        
        
        

In [10]:
class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = Y_train

    #dataset(index)
    def __getitem__(self,index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.n_samples

    #Hyperparameters
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
print(input_size, len(all_words))
print(output_size,tags)
      
dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset, batch_size = batch_size, shuffle = True, num_workers=0)

63 63
7 ['delivery', 'goodbye', 'greeting', 'items', 'payments', 'thanks', 'types']


In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NeuralNet(input_size,hidden_size,output_size).to(device)

#loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    for(words,labels) in train_loader:
        words = words.to(device)
        labels = labels.long()

        #forward
        outputs = model(words)
        #if y would be one-hot,we must apply
        #labels = torch.max(labels,1)
        loss = criterion(outputs, labels)

        #backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if(epoch+1)%100 ==0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print(f'final loss: {loss.item():.4f}')

Epoch [100/1000], Loss: 1.0105
Epoch [200/1000], Loss: 0.1306
Epoch [300/1000], Loss: 0.0169
Epoch [400/1000], Loss: 0.0130
Epoch [500/1000], Loss: 0.0016
Epoch [600/1000], Loss: 0.0013
Epoch [700/1000], Loss: 0.0022
Epoch [800/1000], Loss: 0.0007
Epoch [900/1000], Loss: 0.0009
Epoch [1000/1000], Loss: 0.0006
final loss: 0.0006


In [12]:
data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "hidden_size": hidden_size,
    "output_size": output_size,
    "all_words": all_words,
    "tags": tags
}

FILE = "data.pth"
torch.save(data,FILE)

print(f'training complete. file saved to {FILE}')

training complete. file saved to data.pth


In [36]:
#training part completed..........chat.py started
import random
import json
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('h.json','r') as f:
    intents = json.load(f)

FILE = "data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data["all_words"]
tags = data["tags"]
model_state = data["model_state"]

model = NeuralNet(input_size,hidden_size,output_size).to(device)
model.load_state_dict(model_state)
model.eval()

bot_name = "Sam"
def get_response(msg):
    sentence = tokenize(msg)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                return random.choice(intent['responses'])
    
    return "I do not understand..."
    

In [37]:
import tkinter as Tk
from tkinter import *

In [47]:
BG_GRAY = "#F0F8FF"
BG_COLOR = "#000000"
TEXT_COLOR = "#EAECEE"

FONT = "Helvetica 14"
FONT_BOLD = "Helvetica 13 bold"

class ChatApplication:
    
    def __init__(self):
        self.window = Tk()
        self._setup_main_window()
        
    def run(self):
        self.window.mainloop()
        
    def _setup_main_window(self):
        self.window.title("Chat")
        self.window.resizable(width=False, height=False)
        self.window.configure(width=470, height=550, bg=BG_COLOR)
        
        # head label
        head_label = Label(self.window, bg=BG_COLOR, fg=TEXT_COLOR,
                           text="Welcome", font=FONT_BOLD, pady=10)
        head_label.place(relwidth=1)
        
        # tiny divider
        line = Label(self.window, width=450, bg=BG_GRAY)
        line.place(relwidth=1, rely=0.07, relheight=0.012)
        
        # text widget
        self.text_widget = Text(self.window, width=20, height=2, bg=BG_COLOR, fg=TEXT_COLOR,
                                font=FONT, padx=5, pady=5)
        self.text_widget.place(relheight=0.745, relwidth=1, rely=0.08)
        self.text_widget.configure(cursor="arrow", state=DISABLED)
        
        # scroll bar
        scrollbar = Scrollbar(self.text_widget)
        scrollbar.place(relheight=1, relx=0.974)
        scrollbar.configure(command=self.text_widget.yview)
        
        # bottom label
        bottom_label = Label(self.window, bg=BG_GRAY, height=80)
        bottom_label.place(relwidth=1, rely=0.825)
        
        # message entry box
        self.msg_entry = Entry(bottom_label, bg="#2C3E50", fg=TEXT_COLOR, font=FONT)
        self.msg_entry.place(relwidth=0.74, relheight=0.06, rely=0.008, relx=0.011)
        self.msg_entry.focus()
        self.msg_entry.bind("<Return>", self._on_enter_pressed)
        
        # send button
        send_button = Button(bottom_label, text="Send", font=FONT_BOLD, width=20, bg=BG_GRAY,
                             command=lambda: self._on_enter_pressed(None))
        send_button.place(relx=0.77, rely=0.008, relheight=0.06, relwidth=0.22)
     
    def _on_enter_pressed(self, event):
        msg = self.msg_entry.get()
        self._insert_message(msg, "You")
        
    def _insert_message(self, msg, sender):
        if not msg:
            return
        
        self.msg_entry.delete(0, END)
        msg1 = f"{sender}: {msg}\n\n"
        self.text_widget.configure(state=NORMAL)
        self.text_widget.insert(END, msg1)
        self.text_widget.configure(state=DISABLED)
        
        msg2 = f"{bot_name}: {get_response(msg)}\n\n"
        self.text_widget.configure(state=NORMAL)
        self.text_widget.insert(END, msg2)
        self.text_widget.configure(state=DISABLED)
        
        self.text_widget.see(END)
             
        
if __name__ == "__main__":
    app = ChatApplication()
    app.run()